In [61]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import redis
import json
from pprint import pprint

import pymongo
from datetime import datetime, timedelta, date
import pandas as pd


import numpy as np
import pymongo
import cx_Oracle
import os
import time
import plotly.express as px
from datetime import datetime, timedelta, date


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None  # default='warn'

r = redis.Redis(host="127.0.0.1", port=6379, db=0)

In [57]:
def encode(s):
    
    i = 0
    t = ''
    while i < len(s):
        if s[i].isupper():
            t += 'U'
        elif s[i].islower():
            t += 'L'
        elif s[i].isdigit():
            t += 'D'
        else:
            t += 'O'
        i += 1
        
    return t

def short_encode(s):
    i = 0
    s = encode(s)
    curr = ''
    t = ''
    
    while i < len(s):
        if curr != s[i]:
            t += s[i]
            curr = s[i]
        
        i += 1
    return t

def score_sus_email(s):

    email_input = s
    email_input = email_input.split('@')[0]
    
    
    
    pat_count_digit = encode(email_input).count('D')
    pat_encode = short_encode(email_input)
    
    if pat_encode == 'LD' and pat_count_digit >= 4:
        return 0.25
    elif pat_encode == 'D' and pat_count_digit >= 4:
        return 0.25
    return 0
    
score_sus_email('qywyeqeyqw123@gmail.com')

0

In [58]:
from gibberish_detector import detector
Detector = detector.create_from_model(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/gibberish-detector/big.model')
from nostril import nonsense


def score_gibberish(input_email):
    split_email = input_email.split('@')[0]
    #print(split_email)

    flag_gibberish1 = False
    
    try:
        if Detector.is_gibberish(split_email):
            flag_gibberish1 = True
    except Exception as e:
        pass
    
    flag_gibberish2 = False
    
    try:
        if nonsense(split_email):
            flag_gibberish2 = True
    except Exception as e:
        pass


    if flag_gibberish1 or flag_gibberish2:
        return 0.5
    else:
        return 0

email = 'satyaaji489@gmail.com'
score_gibberish(email)

0.5

In [6]:
alfagift_member = pd.read_json('/home/server/mongo_export/alfagift_member.json')
alfagift_member['sapa'] = alfagift_member['sapa'].fillna('-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: x.get('store_code') \
                                                        if x != '-' else '-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: '-' \
                                                        if x == '' else x)

alfagift_member['created_at'] = alfagift_member['created_at'].fillna('2019-01-01T23:08:53.393Z')
alfagift_member['created_at'] = alfagift_member['created_at'].apply(lambda x: x.get('$date') \
                                                                    if type(x) != str else x)

# alfagift_member = alfagift_member[['account_card', 'sapa', 'created_at']]
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: 'sapa' if x != '-' else 'not_sapa')

alfagift_member['created_at_d'] = pd.to_datetime(alfagift_member['created_at']).dt.strftime("%Y-%m-%d")
alfagift_member['created_at_m'] = pd.to_datetime(alfagift_member['created_at']).dt.strftime("%Y-%m")

alfagift_member['account_card'] = alfagift_member['account_card'].astype('str')

In [12]:
## creating label for training set

alfagift_member_sel = alfagift_member[(alfagift_member['created_at_m'] == '2022-01') |
                                      (alfagift_member['created_at_m'] == '2022-02') |
                                      (alfagift_member['created_at_m'] == '2022-03') |
                                      (alfagift_member['created_at_m'] == '2022-04') ]
alfagift_member_sel['created_at_d'] = pd.to_datetime(alfagift_member_sel['created_at']).dt.strftime("%Y-%m-%d")


li_sus = []
for dr in pd.date_range('2022-01-01', '2022-04-06', freq='D'):
    sel_created = dr.strftime("%Y-%m-%d")

    alfagift_member_sel_d = alfagift_member_sel[alfagift_member_sel['created_at_d'] == sel_created]
    #print(alfagift_member_sel_d.shape)


    ## suspect by IP
    alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
                            .agg(createdFromIp_count=('createdFromIp','count'))
    alfagift_member_sel_d_ip = alfagift_member_sel_d_ip[
        alfagift_member_sel_d_ip['createdFromIp_count'] >= 5].reset_index()

    sus_by_ip = pd.merge(alfagift_member_sel_d_ip, alfagift_member_sel_d, on='createdFromIp', how='left')

    ## suspect by phone
    alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
    alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
                                .agg(createdFromIp_count=('phone_sub','count'))
    alfagift_member_sel_d_phone = alfagift_member_sel_d_phone[
        alfagift_member_sel_d_phone['createdFromIp_count'] >= 5].reset_index()

    sus_by_phone = pd.merge(alfagift_member_sel_d_phone, alfagift_member_sel_d, on='phone_sub', how='left')
    
    ## saving by concating suspect aspect
    sus_all = pd.concat([sus_by_ip[['account_card']], sus_by_phone[['account_card']]], axis=0)\
        .drop_duplicates()
    li_sus.append(sus_all)
    sus_all.to_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud/sus_{}'.format(sel_created))

df_sus_all = pd.concat(li_sus).drop_duplicates()
df_sus_all.to_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud/sus_all.csv', index=False)

In [63]:
li_sus = []
for dr in pd.date_range('2022-01-01', '2022-04-06', freq='D'):
    sel_created = dr.strftime("%Y-%m-%d")

    alfagift_member_sel_d = alfagift_member_sel[alfagift_member_sel['created_at_d'] == sel_created]


    dict_register_ip = {}
    r.set("register_ip", json.dumps(dict_register_ip))
    dict_register_prefix_phone = {}
    r.set("register_prefix_phone", json.dumps(dict_register_prefix_phone))
    dict_register_lower_name = {}
    r.set("register_lower_name", json.dumps(dict_register_lower_name))

    li_score_ip = []
    li_score_prefix_phone = []
    li_score_full_name = []

    for idx, row in alfagift_member_sel_d.iterrows():
        cur_ip = row['createdFromIp']
        cur_prefix_phone = row['phone'][0:9]
        cur_lower_name = row['full_name'].lower()

        student_obj_as_bytes = r.get("register_ip")
        student_obj_as_str = student_obj_as_bytes.decode("utf-8")
        register_ip = json.loads(student_obj_as_str)

        student_obj_as_bytes = r.get("register_prefix_phone")
        student_obj_as_str = student_obj_as_bytes.decode("utf-8")
        register_prefix_phone = json.loads(student_obj_as_str)

        student_obj_as_bytes = r.get("register_lower_name")
        student_obj_as_str = student_obj_as_bytes.decode("utf-8")
        register_lower_name = json.loads(student_obj_as_str)

        score_ip = 0
        if cur_ip in register_ip:
            score_ip += 1
        score_prefix_phone = 0
        if cur_prefix_phone in register_prefix_phone:
            score_prefix_phone += 1
        score_full_name = 0
        if cur_lower_name in register_lower_name:
            score_full_name += 1

        li_score_ip.append(score_ip)
        li_score_prefix_phone.append(score_prefix_phone)
        li_score_full_name.append(score_full_name)


        #print(register_ip) 
        if cur_ip in register_ip:
            register_ip[cur_ip] += 1
        else:
            register_ip[cur_ip] = 1

        if cur_prefix_phone in register_prefix_phone:
            register_prefix_phone[cur_prefix_phone] += 1
        else:
            register_prefix_phone[cur_prefix_phone] = 1

        if cur_lower_name in register_lower_name:
            register_lower_name[cur_lower_name] += 1
        else:
            register_lower_name[cur_lower_name] = 1

        r.set("register_ip", json.dumps(register_ip))
        r.set("register_prefix_phone", json.dumps(register_prefix_phone))
        r.set("register_lower_name", json.dumps(register_lower_name))

        break

In [ ]:
df_sample_test['score_ip'] = li_score_ip
df_sample_test['score_prefix_phone'] = li_score_prefix_phone
df_sample_test['score_full_name'] = li_score_full_name
df_sample_test['score_gibberish'] = df_sample_test['email'].apply(score_gibberish)
df_sample_test['score_sus_email'] = df_sample_test['email'].apply(score_sus_email)

In [ ]:
# df_sample_test.head(100)

In [ ]:
df_sample_test[df_sample_test['createdFromIp'].str.contains('125.166.227.104', case=False)]

In [ ]:
df_sample_test[df_sample_test['score_ip'] >= 0.25]\
[['full_name','phone','email','createdFromIp','score_prefix_phone','score_ip',
  'score_full_name','score_gibberish','score_sus_email']].head(100)